# AWS Bedrock
## Text Generation

In this notebook, we will explore the use of the AWS Bedrock Converse API, an interface that allows conversational interaction with foundational language models (FLMs) hosted on the Amazon Bedrock service.

The goal is to demonstrate how to establish a connection with Bedrock, send messages or prompts to the model, and process responses within a conversational flow.

# 1- Text Generation

Learn the basics of the Amazon Bedrock Invoke API

In [1]:
import json
import boto3
import botocore
from IPython.display import display, Markdown
import time

In [3]:
# Init Bedrock client
session = boto3.session.Session()
region = session.region_name
bedrock = boto3.client(service_name='bedrock-runtime', region_name=region)

In [27]:
model_sonnet_id = "eu.anthropic.claude-3-7-sonnet-20250219-v1:0"

In [6]:
# Create prompt 
city = "Valencia"

prompt = f"""Create a three-day itinerary to explore the city. The city is: 
<text>
{city}
</text>
"""

## Invoke Model

In [7]:
# Create request body for Claude 3.7 Sonnet
claude_body = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.9,
    "messages": [
        {
            "role": "user",
            "content": [{"type": "text", "text": prompt}]
        }
    ],
})

In [9]:
response = bedrock.invoke_model(
        modelId=model_sonnet_id,
        body=claude_body,
        accept="application/json",
        contentType="application/json"
    )

response_body = json.loads(response.get('body').read())

In [ ]:
response_body

Although the Invoke Model API allows direct access to base models, it has several limitations:

- It uses different request/response formats for each model family.

- It does not offer built-in support for multi-turn conversations.

- It requires custom handling based on the capabilities of each model.

## Converse

In [14]:
# Create a converse request 
converse_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.4,
        "topP": 0.9,
        "maxTokens": 500
    }
}

In [15]:
response = bedrock.converse(
        modelId=model_sonnet_id,
        messages=converse_request["messages"],
        inferenceConfig=converse_request["inferenceConfig"]
    )

In [ ]:
response

The Converse API facilitates multi-turn conversations. We can extract the response directly and simulate the conversation:

In [18]:
model_response = response["output"]["message"]["content"][0]["text"]

## Multi-Turn conversations

In [20]:
# create conversation
multi_turn_messages = [
    {
        "role": "user",
        "content": [{"text": prompt}]
    },
    {
        "role": "assistant",
        "content": [{"text": model_response}]
    },
    {
        "role": "user",
        "content": [{"text": "Only a two-day trip."}]
    }
]


In [21]:
response = bedrock.converse(
        modelId=model_sonnet_id,
        messages=multi_turn_messages,
        inferenceConfig={"temperature": 0.2, "maxTokens": 500}
    )

In [ ]:
response

## Streaming

Both InvokeModel and Converse allow streaming responses.

In [23]:
def stream_converse(model_id, messages, inference_config=None):
    if inference_config is None:
        inference_config = {}
    
    print("Streaming response (chunks will appear as they are received):\n")
    print("-" * 80)
    
    full_response = ""
    
    try:
        response = bedrock.converse_stream(
            modelId=model_id,
            messages=messages,
            inferenceConfig=inference_config
        )
        response_stream = response.get('stream')
        if response_stream:
            for event in response_stream:

                if 'messageStart' in event:
                    print(f"\nRole: {event['messageStart']['role']}")

                if 'contentBlockDelta' in event:
                    print(event['contentBlockDelta']['delta']['text'], end="")

                if 'messageStop' in event:
                    print(f"\nStop reason: {event['messageStop']['stopReason']}")

                if 'metadata' in event:
                    metadata = event['metadata']
                    if 'usage' in metadata:
                        print("\nToken usage")
                        print(f"Input tokens: {metadata['usage']['inputTokens']}")
                        print(
                            f":Output tokens: {metadata['usage']['outputTokens']}")
                        print(f":Total tokens: {metadata['usage']['totalTokens']}")
                    if 'metrics' in event['metadata']:
                        print(
                            f"Latency: {metadata['metrics']['latencyMs']} milliseconds")

                
            print("\n" + "-" * 80)
        return full_response
    
    except Exception as e:
        print(f"Error in streaming: {str(e)}")
        return None

In [24]:
streaming_request = [
    {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ]
    }
]

In [ ]:
streamed_response = stream_converse(
    model_sonnet_id, 
    streaming_request, 
    inference_config={"temperature": 0.4, "maxTokens": 1000}
)

## Function Calling

Podemos configurar un LLM para llamar a una función o tool en su conversación. Esto se realiza a través de *toolConfig*

In [28]:
def get_weather(location):
    """Mock function that would normally call a weather API"""
    print(f"Looking up weather for {location}...")
  
    # In a real application, this would call a weather API
    weather_data = {
        "New York": {"condition": "Partly Cloudy", "temperature": 72, "humidity": 65},
        "San Francisco": {"condition": "Foggy", "temperature": 58, "humidity": 80},
        "Miami": {"condition": "Sunny", "temperature": 85, "humidity": 75},
        "Seattle": {"condition": "Rainy", "temperature": 52, "humidity": 90}
    }
  
    return weather_data.get(location, {"condition": "Unknown", "temperature": 0, "humidity": 0})

In [29]:
# define our tool

weather_tool = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_weather",
                "description": "Get current weather for a specific location",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city name to get weather for"
                            }
                        },
                        "required": ["location"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}  # Let the model decide when to use the tool
    }
}

In [30]:
# model request

function_request = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "What's the weather like in San Francisco right now? And what should I wear?"
                }
            ]
        }
    ],
    "inferenceConfig": {
        "temperature": 0.0,  # Use 0 temperature for deterministic function calling
        "maxTokens": 500
    }
}

In [ ]:
response = bedrock.converse(
    modelId=MODELS["Claude 3.7 Sonnet"],
    messages=function_request["messages"],
    inferenceConfig=function_request["inferenceConfig"],
    toolConfig=weather_tool
)

In [ ]:
response

We can see that it detects a toolUse. We could generate a function to execute this tool:

In [34]:
content_blocks = response["output"]["message"]["content"]
has_tool_use = any("toolUse" in block for block in content_blocks)

In [41]:
content_blocks[1]['toolUse']

{'toolUseId': 'tooluse_bklY_OqiTAGfyEJlqmo6ew',
 'name': 'get_weather',
 'input': {'location': 'San Francisco'}}

In [43]:
# extract needed data to execute
if has_tool_use:
    tool_use = content_blocks[1]['toolUse']
    tool_name = tool_use["name"]
    tool_input = tool_use["input"]
    tool_use_id = tool_use["toolUseId"]

In [45]:
tool_output = get_weather(tool_input["location"])

Looking up weather for San Francisco...


Una vez tenemos la salida de la tool podemos pasarlo al llm:

In [47]:
updated_messages = function_request["messages"] + [
                {
                    "role": "assistant",
                    "content": [
                        {
                            "toolUse": {
                                "toolUseId": tool_use_id,
                                "name": tool_name,
                                "input": tool_input
                            }
                        }
                    ]
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use_id,
                                "content": [
                                    {
                                        "json": tool_output
                                    }
                                ],
                                "status": "success"
                            }
                        }
                    ]
                }
            ]

In [50]:
final_response = bedrock.converse(
                modelId=model_sonnet_id,
                messages=updated_messages,
                inferenceConfig=function_request["inferenceConfig"],
                toolConfig=weather_tool  
            )

In [ ]:
final_response

# Multimodal 

In [ ]:
def image_conversation(bedrock_client,
                          model_id,
                          input_text,
                          input_image):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        input text : The input message.
        input_image : The input image.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    print(f"Generating message with model {model_id}")

    # Message to send.

    with open(input_image, "rb") as f:
        image = f.read()

    message = {
        "role": "user",
        "content": [
            {
                "text": input_text
            },
            {
                    "image": {
                        "format": 'jpeg',
                        "source": {
                            "bytes": image
                        }
                    }
            }
        ]
    }

    messages = [message]

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages
    )

    return response